<a href="https://colab.research.google.com/github/marek-bardonski/airev-advanced-workshops/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip3 install wget
print ("Hello world!")
import wget
from zipfile import ZipFile

Hello world!


In [0]:
print('Beginning dataset download with wget module')

url = 'https://bardonski.pl/chineese.zip'
wget.download(url)
url = 'https://bardonski.pl/arabic.zip'
wget.download(url)

In [0]:
zip = ZipFile('chineese.zip')
zip.extractall()

!mkdir chineese
zip = ZipFile('630_webhose-2016-10_20170904084325.zip')
zip.extractall('chineese')

630_webhose-2016-10_20170904084325.zip	chineese      sample_data
arabic.zip				chineese.zip


In [0]:
zip = ZipFile('arabic.zip')
zip.extractall()

!mkdir arabic

In [0]:
zip = ZipFile('627_webhose-2016-10_20170904083346.zip')
zip.extractall('arabic')


In [27]:
!ls


627_webhose-2016-10_20170904083346.zip	arabic	    chineese	  sample_data
630_webhose-2016-10_20170904084325.zip	arabic.zip  chineese.zip
